<a href="https://colab.research.google.com/github/rubycho/cv-notes/blob/master/CV03/CV03-O%20Detecting%20in%20video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (Optional) Detecting in video, using darknet

In this file, we will detect objects in video, using darknet + colab.

Colab doesn't have a display, so we will modify darknet c codes to save the detection results to multiple images.

And then we will convert those to video.

To run this file
- you need pretrained weights from CV03-3 (`yolov3_ts_train_900.weights`).
- also kaggle credentials used on CV03-3.

### Download Darknet

In [1]:
# clone darknet and build
!git clone https://github.com/pjreddie/darknet.git
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 20.53 MiB/s, done.
Resolving deltas: 100% (3918/3918), done.
/content/darknet


**IMPORTANT (same as CV03-3)**

Please check the items below; or training will be really slow.

1. You should run colab notebook as **GPU** mode.
1. Before make, set GPU=1, OPENCV=1 on `Makefile`.

Now upload weights to darknet directory.

### Prepare dataset

In [ ]:
%env KAGGLE_USERNAME=USERNAME_WITHOUT_QUOTES
%env KAGGLE_KEY=KEY_WITHOUT_QUOTES

!kaggle datasets download -d valentynsichkar/traffic-signs-dataset-in-yolo-format
!unzip -d dataset -q traffic-signs-dataset-in-yolo-format.zip

!cp dataset/yolov3_ts_test.cfg ./cfg/
!cp dataset/yolov3_ts_train.cfg ./cfg/

!mkdir ts
!cp -rf dataset/ts/ts/* ./ts/

In [3]:
def adjust_path(fpath: str, old: str, new: str):
  content = []
  with open(fpath, 'r') as f:
    for line in f.readlines():
      content.append(line.replace(old, new))

  with open(fpath, 'w') as f:
    f.write(''.join(content))

OLD_NAME = '/home/my_name/'
NEW_NAME = '/content/darknet/'

adjust_path('./dataset/train.txt', OLD_NAME, NEW_NAME)
adjust_path('./dataset/test.txt', OLD_NAME, NEW_NAME)
adjust_path('./dataset/ts_data.data', OLD_NAME, NEW_NAME)

In [4]:
!cp ./dataset/classes.names ./ts/
!cp ./dataset/train.txt ./ts/
!cp ./dataset/test.txt ./ts/
!cp ./dataset/ts_data.data ./ts/

# change coco.names
!cp -rf ./dataset/classes.names ./data/coco.names

### Modify c code

Now, download `demo.c`, `image.c` from `src` directory.

In `demo.c`:

- comment out conditional statements on `display_in_thread()`.
  ```c
  void *display_in_thread(void *ptr)
  {
      int c = show_image(buff[(buff_index + 1)%3], "Demo", 1);
      // if (c != -1) c = c%256;
      // if (c == 27) {
      //    demo_done = 1;
      //    return 0;
      // } else if (c == 82) {
      //    demo_thresh += .02;
      // } else if (c == 84) {
      //    demo_thresh -= .02;
      //    if(demo_thresh <= .02) demo_thresh = .02;
      // } else if (c == 83) {
      //    demo_hier += .02;
      // } else if (c == 81) {
      //    demo_hier -= .02;
      //    if(demo_hier <= .0) demo_hier = .0;
      // }
      return 0;
  }
  ```
- comment out `make_window()` on `demo()`.
  ```c
    ...

    buff[0] = get_image_from_stream(cap);
    buff[1] = copy_image(buff[0]);
    buff[2] = copy_image(buff[0]);
    buff_letter[0] = letterbox_image(buff[0], net->w, net->h);
    buff_letter[1] = letterbox_image(buff[0], net->w, net->h);
    buff_letter[2] = letterbox_image(buff[0], net->w, net->h);

    int count = 0;
    if(!prefix){
        // make_window("Demo", 1352, 1013, fullscreen);
    }

    demo_time = what_time_is_it_now();

    ...
  ```

In `image.c`:

- add static variable on top:
  ```c
  static int idx = 0;
  ```
- change `show_image()` as below:
  ```c
  int show_image(image p, const char *name, int ms)
  {
      char buf[50];
      sprintf(buf, "predictions%02d", idx++);

      fprintf(stderr, "saving to %s.jpg\n", buf);
      save_image(p, buf);
      return -1;
  }
  ```
- modify `get_label()` and `draw_label()` as below:
  ```c
  image get_label(image **characters, char *string, int size)
  {
    size = 1;
    // size = size / 10;
    // if (size > 7) size = 7;
    image label = make_empty_image(0,0,0);
    while(*string){
    ...
  ```

  ```c
  void draw_label(image a, int r, int c, image label, const float *rgb)
  {
    int w = label.w;
    int h = label.h;
    // if (r - h >= 0) r = r - h;
    if (c - w >= 0) c = c - w;

    int i, j, k;
    ...
  ```
- modify `draw_detections()` as below:
  ```c
  void draw_detections(image im, detection *dets, int num, float thresh, char **names, image **alphabet, int classes)
  {
    ...
            if (dets[i].prob[j] > thresh){
                if (class < 0) {
                    strcat(labelstr, names[j]);
                    sprintf(labelstr, "%s: %.2f", labelstr, dets[i].prob[j] * 100); // print prob
                    class = j;
                } else {
                    strcat(labelstr, ", ");
                    strcat(labelstr, names[j]);
                }
                printf("%s: %.0f%%\n", names[j], dets[i].prob[j]*100);
            }
    ...
  }
  ```


Erase original files on colab, upload modified files, and make.

In [12]:
!make
!./darknet detector demo dataset/ts_data.data cfg/yolov3_ts_test.cfg yolov3_ts_train_900.weights dataset/traffic-sign-to-test.mp4

gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofa

If you have problem with `IplImage` while compiling, download `image_opencv.cpp` and modify `mat_to_image` as below:

```c
image mat_to_image(Mat m)
{
    IplImage ipl = cvIplImage(m);
    // IplImage ipl = m;
    image im = ipl_to_image(&ipl);
    rgbgr_image(im);
    return im;
}
```

In [13]:
!ffmpeg -framerate 25 -i predictions%02d.jpg -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Download output.mp4 file and play.

It will look like this [video](https://github.com/rubycho/cv-notes/blob/master/CV03/output.mp4).

Well, the performance seems to be not that good.

### More information?

- Issue about saving demo results of darknet to file: https://github.com/pjreddie/darknet/issues/102
- Converting images to mp4: https://askubuntu.com/questions/610903/how-can-i-create-a-video-file-from-a-set-of-jpg-images